<a href="https://colab.research.google.com/github/wiktorialasek/Thesis_v2/blob/commit1/export_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
!pip -q install tzdata pyarrow

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [66]:
%%time
# --- parametry ---
TWEET_ID = "1256239815256797184"   # <-- tu wpisujesz swój id
PRICES_MIN = "2010-06-29 21:00:00+00:00"
PRICES_MAX = "2025-03-07 20:54:00+00:00"
DATA_DIR   = "/content/drive/MyDrive/Thesis_v2"
TWEETS_CSV = f"{DATA_DIR}/all_musk_posts.csv"
PRICES_DIR = f"{DATA_DIR}/TSLA_sorted"

from zoneinfo import ZoneInfo
import pandas as pd, os

DISPLAY_TZ = ZoneInfo("Europe/Warsaw")

# wczytaj tweety (z filtrem czasu jak w app.py) using load_tweets function
tweets = load_tweets(TWEETS_CSV)

tweets["tweet_id"] = tweets["tweet_id"].astype(str)
sel = tweets[tweets["tweet_id"] == str(TWEET_ID)].copy()

# najprościej:
print("tweets.in_range:", len(tweets))

print("selected rows:", len(sel))
if sel.empty:
    raise SystemExit("Nie znaleziono tweeta w tym zakresie czasu.")

tweet_dt_utc = pd.Timestamp(sel.iloc[0]["created_at"])
tweet_dt_loc = tweet_dt_utc.tz_convert(DISPLAY_TZ)
print("Wybrany tweet:", TWEET_ID)
print("UTC :", tweet_dt_utc)
print("Lokalnie:", tweet_dt_loc)

# okno 0..60 min
start_utc = tweet_dt_utc.floor("min")
end_utc   = start_utc + pd.Timedelta(minutes=60)
print("Okno cen:", start_utc, "→", end_utc)

tweets.in_range: 53009
selected rows: 1
Wybrany tweet: 1256239815256797184
UTC : 2020-05-01 15:11:26+00:00
Lokalnie: 2020-05-01 17:11:26+02:00
Okno cen: 2020-05-01 15:11:00+00:00 → 2020-05-01 16:11:00+00:00
CPU times: user 1.47 s, sys: 88.4 ms, total: 1.56 s
Wall time: 1.78 s


In [59]:
%%time
import glob, re, os
from datetime import datetime, timezone

assert os.path.isdir(PRICES_DIR), f"Brak katalogu: {PRICES_DIR}"
all_files = sorted(glob.glob(os.path.join(PRICES_DIR, "**", "*.csv"), recursive=True))
print("Liczba plików w TSLA_sorted:", len(all_files))

# Spróbujmy wyciągnąć YYYY-MM-DD z nazwy pliku
date_pat = re.compile(r"(20\d{2}-\d{2}-\d{2})")
start_day = start_utc.date()
end_day   = end_utc.date()

candidates = []
no_date = []
for p in all_files:
    m = date_pat.search(os.path.basename(p))
    if m:
        d = datetime.fromisoformat(m.group(1)).date()
        # kandydatem jest plik, którego data mieści się w {start_day, end_day}
        if d >= start_day and d <= end_day:
            candidates.append(p)
    else:
        no_date.append(p)

print("kandydaci (z datą w nazwie):", len(candidates))
print("bez daty w nazwie:", len(no_date))

# Heurystyka: jeśli znaleźliśmy kandydatów z datą – użyj ich; w przeciwnym razie czytaj wszystko (wolniej)
files_to_read = candidates if candidates else all_files
print("DO ODCZYTU:", len(files_to_read), "plików")
for p in files_to_read[:10]:  # pokaż max 10
    print("  ", os.path.basename(p))


Liczba plików w TSLA_sorted: 3694
kandydaci (z datą w nazwie): 1
bez daty w nazwie: 0
DO ODCZYTU: 1 plików
   2020-05-01.csv
CPU times: user 261 ms, sys: 494 ms, total: 755 ms
Wall time: 2.76 s


In [82]:
%%time
import pandas as pd, time

def to_utc(series, source_tz: str):
    from zoneinfo import ZoneInfo
    s = pd.to_datetime(series, errors="coerce", utc=False)
    try:
        has_tz = s.dt.tz is not None
    except Exception:
        has_tz = False
    if has_tz:
        return s.dt.tz_convert("UTC")
    tz = ZoneInfo("Europe/Warsaw")
    s = s.dt.tz_localize(tz, nonexistent="shift_forward", ambiguous="NaT")
    return s.dt.tz_convert("UTC")

def _pick(raw, col):
    if col in raw.columns: return raw[col]
    if col.capitalize() in raw.columns: return raw[col.capitalize()]
    if col.upper() in raw.columns: return raw[col.upper()]
    raise KeyError(col)

t0 = time.time()
frames = []
for i, path in enumerate(files_to_read, 1):
    # Szybciej: usecols + pyarrow, ale fallback jeśli brak
    try:
        raw = pd.read_csv(
            path,
            engine="pyarrow",
            usecols=lambda c: c.lower() in ("datetime","time","timestamp","date","open","high","low","close"),
        )
    except Exception:
        raw = pd.read_csv(
            path,
            low_memory=False,   # <<< zostaje tylko przy domyślnym parserze (c engine)
            usecols=lambda c: c.lower() in ("datetime","time","timestamp","date","open","high","low","close"),
        )

    dt_col = next((c for c in ["datetime","time","timestamp","date","Date","Time"] if c in raw.columns), None)
    if not dt_col:
        continue

    part = pd.DataFrame({
        "datetime": to_utc(raw[dt_col], "Europe/Warsaw"),
        "open":  pd.to_numeric(_pick(raw,"open"),  errors="coerce"),
        "high":  pd.to_numeric(_pick(raw,"high"),  errors="coerce"),
        "low":   pd.to_numeric(_pick(raw,"low"),   errors="coerce"),
        "close": pd.to_numeric(_pick(raw,"close"), errors="coerce"),
    }).dropna(subset=["datetime"])

    m = part[(part["datetime"] >= start_utc) & (part["datetime"] <= end_utc)]
    if not m.empty:
        frames.append(m)

prices_win = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame(columns=["datetime","open","high","low","close"])
prices_win = prices_win.sort_values("datetime").reset_index(drop=True)

print(f"prices_win rows: {len(prices_win)}  (okno: {start_utc} → {end_utc})  czas czytania: {time.time()-t0:.2f}s")
if not prices_win.empty:
    print("zakres wczytanych cen:", prices_win["datetime"].min(), "→", prices_win["datetime"].max())
    display(prices_win.head(3))


prices_win rows: 61  (okno: 2020-05-01 15:11:00+00:00 → 2020-05-01 16:11:00+00:00)  czas czytania: 0.04s
zakres wczytanych cen: 2020-05-01 15:11:00+00:00 → 2020-05-01 16:11:00+00:00


,datetime,open,high,low,close
0,2020-05-01 15:11:00+00:00,253.33,253.33,250.07,250.10
1,2020-05-01 15:12:00+00:00,250.07,250.76,247.68,248.72
2,2020-05-01 15:13:00+00:00,248.72,248.77,247.04,248.49


CPU times: user 24.3 ms, sys: 3.1 ms, total: 27.4 ms
Wall time: 49.5 ms


In [61]:
%%time
if prices_win.empty:
    minute_open = pd.Series(dtype=float)
else:
    df = prices_win.copy()
    df["minute"] = df["datetime"].dt.floor("min")
    per_min = df.sort_values("datetime").groupby("minute").last()
    minute_open = per_min["open"].astype(float)
    minute_open.index = pd.to_datetime(minute_open.index)

print("minute_open len:", len(minute_open))
if len(minute_open):
    print("minute_open range:", minute_open.index.min(), "→", minute_open.index.max())

base_minute = start_utc  # to jest floor("min") już wyliczony wcześniej
base_open = minute_open.get(base_minute, None)
print("base_minute:", base_minute, " base_open:", base_open)


minute_open len: 61
minute_open range: 2020-05-01 15:11:00+00:00 → 2020-05-01 16:11:00+00:00
base_minute: 2020-05-01 15:11:00+00:00  base_open: 253.33
CPU times: user 4.47 ms, sys: 55 µs, total: 4.52 ms
Wall time: 4.36 ms


In [65]:
%%time
# === Komórka E — liczenie i finalna tabela w żądanej kolejności ===

# 1..20, 30, 60
INTERVALS = tuple(list(range(1,21)) + [30,60])

def pct_changes_for_tweet(minute_series, tweet_dt_utc, intervals):
    base_minute = pd.Timestamp(tweet_dt_utc).floor("min")
    base = minute_series.get(base_minute, None)
    out = {}
    for m in intervals:
        tmin = base_minute + pd.Timedelta(minutes=m)
        p = minute_series.get(tmin, None)
        out[m] = round((p - base) / base * 100.0, 2) if (base is not None and p is not None and base != 0) else None
    return out, base

if base_open is None:
    print("Brak notowań (OPEN) w minucie tweeta — nie ma z czego liczyć.")
    res = pd.DataFrame()
else:
    pct, base_open_check = pct_changes_for_tweet(minute_open, tweet_dt_utc, INTERVALS)
    # zbuduj pełny wiersz z tekstem
    row = {
        "tweet_id": str(TWEET_ID),
        "text": sel.iloc[0]["text"],   # pełny tekst tweeta
        "created_at_local": tweet_dt_loc.strftime("%Y-%m-%d %H:%M:%S %Z"),
        "price_at_tweet_open": base_open_check,
        **{f"change_{m}m": pct[m] for m in INTERVALS}
    }
    res = pd.DataFrame([row])

if res.empty:
    print("Brak wyniku.")
else:
    # kolejność kolumn dokładnie wg Twojego życzenia:
    change_cols = [f"change_{m}m" for m in list(range(1,21)) + [30,60]]
    col_order = ["tweet_id", "text", "created_at_local", "price_at_tweet_open"] + change_cols
    # dla bezpieczeństwa – pokaż tylko kolumny, które istnieją
    col_order = [c for c in col_order if c in res.columns]
    display(res[col_order])


,tweet_id,text,created_at_local,price_at_tweet_open,change_1m,change_2m,change_3m,change_4m,change_5m,change_6m,...,change_13m,change_14m,change_15m,change_16m,change_17m,change_18m,change_19m,change_20m,change_30m,change_60m
0,1256239815256797184,Tesla stock price is too high imo,2020-05-01 17:11:26 CEST,253.33,-1.29,-1.82,-1.91,-1.58,-2.89,-3.55,...,-4.59,-4.45,-4.4,-3.79,-3.7,-3.18,-3.66,-4.44,-6.84,-8.74


CPU times: user 15.5 ms, sys: 0 ns, total: 15.5 ms
Wall time: 15.4 ms


In [67]:
!pip -q install tzdata pyarrow
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [68]:
# --- parametry ---
TWEET_ID   = "1256239815256797184"          # <- Twój ID
DATA_DIR   = "/content/drive/MyDrive/Thesis_v2"
TWEETS_CSV = f"{DATA_DIR}/all_musk_posts.csv"
PRICES_DIR = f"{DATA_DIR}/TSLA_sorted"

# stały filtr (jak w app.py)
PRICES_MIN = "2010-06-29 21:00:00+00:00"
PRICES_MAX = "2025-03-07 20:54:00+00:00"

import os, glob, time, re
import pandas as pd
from datetime import datetime
from zoneinfo import ZoneInfo

DISPLAY_TZ = ZoneInfo("Europe/Warsaw")
PRICES_SOURCE_TZ = "Europe/Warsaw"

def to_utc(series, source_tz: str):
    s = pd.to_datetime(series, errors="coerce", utc=False)
    try:
        has_tz = s.dt.tz is not None
    except Exception:
        has_tz = False
    if has_tz:
        return s.dt.tz_convert("UTC")
    tz = ZoneInfo(source_tz)
    s = s.dt.tz_localize(tz, nonexistent="shift_forward", ambiguous="NaT")
    return s.dt.tz_convert("UTC")

def load_tweets(csv_path: str) -> pd.DataFrame:
    """Wczytaj tweety i przefiltruj po PRICES_MIN..PRICES_MAX (UTC), jak w app.py."""
    assert os.path.exists(csv_path), f"Brak pliku: {csv_path}"
    df = pd.read_csv(csv_path, low_memory=False)
    df["tweet_id"] = df["id"] if "id" in df.columns else range(1, len(df)+1)
    df["text"] = df["fullText"].fillna("") if "fullText" in df.columns else df.get("text", "").fillna("")
    assert "createdAt" in df.columns, "Brakuje kolumny 'createdAt'"
    df["created_at"] = pd.to_datetime(df["createdAt"], errors="coerce", utc=True)

    lo = pd.to_datetime(PRICES_MIN, utc=True)
    hi = pd.to_datetime(PRICES_MAX, utc=True)
    df = df[(df["created_at"] >= lo) & (df["created_at"] <= hi)]
    df = df.dropna(subset=["created_at"]).sort_values("created_at").reset_index(drop=True)
    return df[["tweet_id","text","created_at"]]


In [69]:
%%time
# wczytaj i wybierz tweeta
tweets = load_tweets(TWEETS_CSV)
tweets["tweet_id"] = tweets["tweet_id"].astype(str)
sel = tweets[tweets["tweet_id"] == str(TWEET_ID)].copy()

print("tweets.in_range:", len(tweets))
print("selected rows:", len(sel))
if sel.empty:
    raise SystemExit("Nie znaleziono tweeta w tym zakresie czasu.")

tweet_dt_utc = pd.Timestamp(sel.iloc[0]["created_at"])
tweet_dt_loc = tweet_dt_utc.tz_convert(DISPLAY_TZ)
print("Wybrany tweet:", TWEET_ID)
print("UTC :", tweet_dt_utc, " | Lokalnie:", tweet_dt_loc)

# okno 0..60 min
start_utc = tweet_dt_utc.floor("min")
end_utc   = start_utc + pd.Timedelta(minutes=60)
print("Okno cen:", start_utc, "→", end_utc)


tweets.in_range: 53009
selected rows: 1
Wybrany tweet: 1256239815256797184
UTC : 2020-05-01 15:11:26+00:00  | Lokalnie: 2020-05-01 17:11:26+02:00
Okno cen: 2020-05-01 15:11:00+00:00 → 2020-05-01 16:11:00+00:00
CPU times: user 1.54 s, sys: 90.2 ms, total: 1.63 s
Wall time: 2.27 s


In [70]:
%%time
# Spróbujmy zawęzić pliki po dacie w nazwie (jeśli nazwy mają YYYY-MM-DD)
assert os.path.isdir(PRICES_DIR), f"Brak katalogu: {PRICES_DIR}"
all_files = sorted(glob.glob(os.path.join(PRICES_DIR, "**", "*.csv"), recursive=True))
print("Liczba plików w TSLA_sorted:", len(all_files))

date_pat = re.compile(r"(20\d{2}-\d{2}-\d{2})")
start_day = start_utc.date()
end_day   = end_utc.date()

candidates, no_date = [], []
for p in all_files:
    m = date_pat.search(os.path.basename(p))
    if m:
        d = datetime.fromisoformat(m.group(1)).date()
        if start_day <= d <= end_day:
            candidates.append(p)
    else:
        no_date.append(p)

files_to_read = candidates if candidates else all_files
print("kandydaci:", len(candidates), "| bez_dat:", len(no_date), "| DO ODCZYTU:", len(files_to_read))

# Czytamy tylko potrzebne kolumny i filtrujemy wiersze do okna czasu
frames = []
t0 = time.time()
for path in files_to_read:
    try:
        raw = pd.read_csv(
            path,
            low_memory=False,
            engine="pyarrow",
            usecols=lambda c: c.lower() in ("datetime","time","timestamp","date","open","high","low","close"),
        )
    except Exception:
        raw = pd.read_csv(
            path,
            low_memory=False,
            usecols=lambda c: c.lower() in ("datetime","time","timestamp","date","open","high","low","close"),
        )
    dt_col = next((c for c in ["datetime","time","timestamp","date","Date","Time"] if c in raw.columns), None)
    if not dt_col:
        continue

    part = pd.DataFrame({
        "datetime": to_utc(raw[dt_col], PRICES_SOURCE_TZ),
        "open":  pd.to_numeric(raw[[c for c in raw.columns if c.lower()=="open"][0]],  errors="coerce"),
        "high":  pd.to_numeric(raw[[c for c in raw.columns if c.lower()=="high"][0]],  errors="coerce"),
        "low":   pd.to_numeric(raw[[c for c in raw.columns if c.lower()=="low"][0]],   errors="coerce"),
        "close": pd.to_numeric(raw[[c for c in raw.columns if c.lower()=="close"][0]], errors="coerce"),
    }).dropna(subset=["datetime"])

    m = part[(part["datetime"] >= start_utc) & (part["datetime"] <= end_utc)]
    if not m.empty:
        frames.append(m)

prices_win = (pd.concat(frames, ignore_index=True)
              if frames else pd.DataFrame(columns=["datetime","open","high","low","close"]))
prices_win = prices_win.sort_values("datetime").reset_index(drop=True)

print(f"prices_win rows: {len(prices_win)} | czas czytania: {time.time()-t0:.2f}s")
if not prices_win.empty:
    print("zakres cen:", prices_win["datetime"].min(), "→", prices_win["datetime"].max())
    display(prices_win.head(3))

# seria minutowa OPEN
if prices_win.empty:
    minute_open = pd.Series(dtype=float)
else:
    df = prices_win.copy()
    df["minute"] = df["datetime"].dt.floor("min")
    per_min = df.sort_values("datetime").groupby("minute").last()
    minute_open = per_min["open"].astype(float)
    minute_open.index = pd.to_datetime(minute_open.index)

print("minute_open len:", len(minute_open))
base_minute = start_utc
base_open = minute_open.get(base_minute, None)
print("base_minute:", base_minute, "| base_open:", base_open)


Liczba plików w TSLA_sorted: 3694
kandydaci: 1 | bez_dat: 0 | DO ODCZYTU: 1
prices_win rows: 61 | czas czytania: 0.01s
zakres cen: 2020-05-01 15:11:00+00:00 → 2020-05-01 16:11:00+00:00


,datetime,open,high,low,close
0,2020-05-01 15:11:00+00:00,253.33,253.33,250.07,250.10
1,2020-05-01 15:12:00+00:00,250.07,250.76,247.68,248.72
2,2020-05-01 15:13:00+00:00,248.72,248.77,247.04,248.49


minute_open len: 61
base_minute: 2020-05-01 15:11:00+00:00 | base_open: 253.33
CPU times: user 308 ms, sys: 359 ms, total: 667 ms
Wall time: 2.64 s


In [71]:
%%time
INTERVALS = tuple(list(range(1,21)) + [30,60])

def pct_changes_for_tweet(minute_series, tweet_dt_utc, intervals):
    base_minute = pd.Timestamp(tweet_dt_utc).floor("min")
    base = minute_series.get(base_minute, None)
    out = {}
    for m in intervals:
        tmin = base_minute + pd.Timedelta(minutes=m)
        p = minute_series.get(tmin, None)
        out[m] = round((p - base) / base * 100.0, 2) if (base is not None and p is not None and base != 0) else None
    return out, base

if base_open is None:
    print("Brak notowań (OPEN) w minucie tweeta — nie ma z czego liczyć.")
    res = pd.DataFrame()
else:
    pct, base_open_check = pct_changes_for_tweet(minute_open, tweet_dt_utc, INTERVALS)
    row = {
        "tweet_id": str(TWEET_ID),
        "text": sel.iloc[0]["text"],   # pełny tekst
        "created_at_local": tweet_dt_loc.strftime("%Y-%m-%d %H:%M:%S %Z"),
        "price_at_tweet_open": base_open_check,
        **{f"change_{m}m": pct[m] for m in INTERVALS}
    }
    res = pd.DataFrame([row])

if res.empty:
    print("Brak wyniku.")
else:
    change_cols = [f"change_{m}m" for m in list(range(1,21)) + [30,60]]
    col_order = ["tweet_id","text","created_at_local","price_at_tweet_open"] + change_cols
    col_order = [c for c in col_order if c in res.columns]
    display(res[col_order])


,tweet_id,text,created_at_local,price_at_tweet_open,change_1m,change_2m,change_3m,change_4m,change_5m,change_6m,...,change_13m,change_14m,change_15m,change_16m,change_17m,change_18m,change_19m,change_20m,change_30m,change_60m
0,1256239815256797184,Tesla stock price is too high imo,2020-05-01 17:11:26 CEST,253.33,-1.29,-1.82,-1.91,-1.58,-2.89,-3.55,...,-4.59,-4.45,-4.4,-3.79,-3.7,-3.18,-3.66,-4.44,-6.84,-8.74


CPU times: user 16.1 ms, sys: 124 µs, total: 16.2 ms
Wall time: 16.6 ms


In [83]:
!pip -q install tzdata pyarrow
from google.colab import drive
drive.mount('/content/drive')

# ===== Parametry =====
DATA_DIR   = "/content/drive/MyDrive/Thesis_v2"
TWEETS_CSV = f"{DATA_DIR}/all_musk_posts.csv"
PRICES_DIR = f"{DATA_DIR}/TSLA_sorted"   # katalog z wieloma plikami 1-min
PRICES_MIN = "2010-06-29 21:00:00+00:00"
PRICES_MAX = "2025-03-07 20:54:00+00:00"

SESSION_LOCAL_START = "15:35"  # lokalnie (Europe/Warsaw)
SESSION_LOCAL_END   = "21:54"  # lokalnie (Europe/Warsaw)
INTERVALS = tuple(list(range(1,21)) + [30,60])

# ile wierszy pokazać w podglądzie
PREVIEW_N = 10

# ===== Importy / setup =====
import os, glob, time, re
import pandas as pd
from datetime import datetime
from zoneinfo import ZoneInfo

DISPLAY_TZ = ZoneInfo("Europe/Warsaw")
PRICES_SOURCE_TZ = "Europe/Warsaw"

def to_utc(series, source_tz: str):
    s = pd.to_datetime(series, errors="coerce", utc=False)
    try:
        has_tz = s.dt.tz is not None
    except Exception:
        has_tz = False
    if has_tz:
        return s.dt.tz_convert("UTC")
    tz = ZoneInfo(source_tz)
    s = s.dt.tz_localize(tz, nonexistent="shift_forward", ambiguous="NaT")
    return s.dt.tz_convert("UTC")

def load_tweets(csv_path: str) -> pd.DataFrame:
    """Jak w app.py + filtr stałego zakresu czasu (UTC)."""
    assert os.path.exists(csv_path), f"Brak pliku: {csv_path}"
    df = pd.read_csv(csv_path, low_memory=False)
    df["tweet_id"] = df["id"] if "id" in df.columns else range(1, len(df)+1)
    df["text"] = df["fullText"].fillna("") if "fullText" in df.columns else df.get("text", "").fillna("")
    assert "createdAt" in df.columns, "Brakuje kolumny 'createdAt'"
    df["created_at"] = pd.to_datetime(df["createdAt"], errors="coerce", utc=True)
    lo = pd.to_datetime(PRICES_MIN, utc=True)
    hi = pd.to_datetime(PRICES_MAX, utc=True)
    df = df[(df["created_at"] >= lo) & (df["created_at"] <= hi)]
    df = df.dropna(subset=["created_at"]).sort_values("created_at").reset_index(drop=True)
    return df[["tweet_id","text","created_at"]]

def load_all_prices(base_dir: str) -> pd.DataFrame:
    """Wczytaj wszystkie ceny (tylko potrzebne kolumny), scalisz i posortuj."""
    assert os.path.isdir(base_dir), f"Brak katalogu: {base_dir}"
    files = sorted(glob.glob(os.path.join(base_dir, "**", "*.csv"), recursive=True))
    assert files, f"Brak CSV w {base_dir}"
    frames = []
    t0 = time.time()
    for path in files:
        try:
            raw = pd.read_csv(
                path,
                engine="pyarrow",
                usecols=lambda c: c.lower() in ("datetime","time","timestamp","date","open","high","low","close"),
            )
        except Exception:
            raw = pd.read_csv(
                path,
                low_memory=False,   # <<< zostaje tylko przy domyślnym parserze (c engine)
                usecols=lambda c: c.lower() in ("datetime","time","timestamp","date","open","high","low","close"),
            )
        dt_col = next((c for c in ["datetime","time","timestamp","date","Date","Time"] if c in raw.columns), None)
        if not dt_col:
            continue
        # normalizacja
        open_col  = [c for c in raw.columns if c.lower()=="open"][0]
        high_col  = [c for c in raw.columns if c.lower()=="high"][0]
        low_col   = [c for c in raw.columns if c.lower()=="low"][0]
        close_col = [c for c in raw.columns if c.lower()=="close"][0]

        part = pd.DataFrame({
            "datetime": to_utc(raw[dt_col], PRICES_SOURCE_TZ),
            "open":  pd.to_numeric(raw[open_col],  errors="coerce"),
            "high":  pd.to_numeric(raw[high_col],  errors="coerce"),
            "low":   pd.to_numeric(raw[low_col],   errors="coerce"),
            "close": pd.to_numeric(raw[close_col], errors="coerce"),
        }).dropna(subset=["datetime"])
        frames.append(part)

    prices = (pd.concat(frames, ignore_index=True)
              .sort_values("datetime").reset_index(drop=True))
    print(f"[prices] rows={len(prices)}  range={prices['datetime'].min()} → {prices['datetime'].max()}  took {time.time()-t0:.1f}s")
    return prices

def build_minute_open_series(prices_df: pd.DataFrame) -> pd.Series:
    """Index = minuta (UTC), value = OPEN dla tej minuty (ostatni tick w minucie)."""
    df = prices_df.copy()
    df["minute"] = df["datetime"].dt.floor("min")
    per_min = df.sort_values("datetime").groupby("minute").last()
    s = per_min["open"].astype(float)
    s.index = pd.to_datetime(s.index)
    return s

def pct_changes_for_tweet(minute_series: pd.Series, tweet_dt_utc: pd.Timestamp, intervals=INTERVALS):
    """% względem OPEN w minucie tweeta; gdy brak danych — None."""
    base_minute = pd.Timestamp(tweet_dt_utc).floor("min")
    base = minute_series.get(base_minute, None)
    out = {}
    for m in intervals:
        p = minute_series.get(base_minute + pd.Timedelta(minutes=m), None)
        out[m] = round((p - base) / base * 100.0, 2) if (base is not None and p is not None and base != 0) else None
    return out, base

def filter_session_tweets(df_tweets: pd.DataFrame,
                          session_local_start="15:35",
                          session_local_end="21:54",
                          display_tz=ZoneInfo("Europe/Warsaw")) -> pd.DataFrame:
    """Zostawia tylko tweety opublikowane lokalnie między HH:MM a HH:MM."""
    df = df_tweets.copy()
    df["created_local"] = df["created_at"].dt.tz_convert(display_tz)
    hhmm = df["created_local"].dt.strftime("%H:%M")
    mask = (hhmm >= session_local_start) & (hhmm <= session_local_end)
    return df[mask].drop(columns=["created_local"])

def required_utc_dates_for_60m(df_tweets: pd.DataFrame) -> set[str]:
    """
    Zwraca zbiór stringów 'YYYY-MM-DD' (UTC) potrzebnych do pokrycia
    minuty tweeta ORAZ minuty do +60 min (może wejść w następną dobę).
    """
    base_days = df_tweets["created_at"].dt.floor("D")
    next_days = (df_tweets["created_at"] + pd.Timedelta(minutes=60)).dt.floor("D")
    days = pd.Index(base_days.tolist() + next_days.tolist()).unique()
    return set(days.strftime("%Y-%m-%d"))

def load_prices_for_dates(base_dir: str, date_strings: set[str]) -> pd.DataFrame:
    """
    Wczytuje tylko te pliki z base_dir, które w nazwie mają 'YYYY-MM-DD' z date_strings.
    Jeśli nie znajdzie żadnego dopasowania po nazwie → fallback: czyta wszystko (wolniej).
    """
    assert os.path.isdir(base_dir), f"Brak katalogu: {base_dir}"
    all_files = sorted(glob.glob(os.path.join(base_dir, "**", "*.csv"), recursive=True))
    if not all_files:
        raise FileNotFoundError(f"Brak CSV w {base_dir}")

    # wybierz pliki, które mają którąś z dat w nazwie
    files_to_read = [p for p in all_files if any(d in os.path.basename(p) for d in date_strings)]

    if not files_to_read:
        print("[warn] Nie znalazłem plików po dacie w nazwie — fallback na WSZYSTKO (wolniej).")
        files_to_read = all_files

    frames = []
    t0 = time.time()
    for path in files_to_read:
        try:
            raw = pd.read_csv(
                path,
                engine="pyarrow",
                usecols=lambda c: c.lower() in ("datetime","time","timestamp","date","open","high","low","close"),
            )
        except Exception:
            raw = pd.read_csv(
                path,
                low_memory=False,   # <<< zostaje tylko przy domyślnym parserze (c engine)
                usecols=lambda c: c.lower() in ("datetime","time","timestamp","date","open","high","low","close"),
            )

        dt_col = next((c for c in ["datetime","time","timestamp","date","Date","Time"] if c in raw.columns), None)
        if not dt_col:
            continue
        # normalizacja nazw kolumn OHLC:
        open_col  = [c for c in raw.columns if c.lower()=="open"][0]
        high_col  = [c for c in raw.columns if c.lower()=="high"][0]
        low_col   = [c for c in raw.columns if c.lower()=="low"][0]
        close_col = [c for c in raw.columns if c.lower()=="close"][0]

        part = pd.DataFrame({
            "datetime": to_utc(raw[dt_col], PRICES_SOURCE_TZ),
            "open":  pd.to_numeric(raw[open_col],  errors="coerce"),
            "high":  pd.to_numeric(raw[high_col],  errors="coerce"),
            "low":   pd.to_numeric(raw[low_col],   errors="coerce"),
            "close": pd.to_numeric(raw[close_col], errors="coerce"),
        }).dropna(subset=["datetime"])
        frames.append(part)

    if not frames:
        return pd.DataFrame(columns=["datetime","open","high","low","close"])

    prices = (pd.concat(frames, ignore_index=True)
              .sort_values("datetime").reset_index(drop=True))
    print(f"[prices/selected] files={len(files_to_read)} rows={len(prices)} "
          f"range={prices['datetime'].min()} → {prices['datetime'].max()} took {time.time()-t0:.1f}s")
    return prices


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [84]:
# ===== 1) Wczytaj tweety i zawęź do godzin sesji =====
tweets_all = load_tweets(TWEETS_CSV)
print("[tweets] po filtrze UTC:", len(tweets_all))

tweets_sess = filter_session_tweets(
    tweets_all,
    session_local_start="15:35",
    session_local_end="21:54",
    display_tz=DISPLAY_TZ
)
print("[tweets] w sesji 15:35–21:54:", len(tweets_sess))

# jeśli jest pusto — przerwij, nie ma co liczyć
if tweets_sess.empty:
    raise SystemExit("Brak tweetów w godzinach sesji w zadanym zakresie dat.")

[tweets] po filtrze UTC: 53009
[tweets] w sesji 15:35–21:54: 16944


In [85]:
# ===== 2) Wyznacz potrzebne dni (UTC) i wczytaj tylko te ceny =====
need_days = required_utc_dates_for_60m(tweets_sess)
print("[dates] potrzeba dni UTC:", sorted(list(need_days))[:10], "… (łączna liczba:", len(need_days), ")")

prices_sel = load_prices_for_dates(PRICES_DIR, need_days)
if prices_sel.empty:
    raise SystemExit("Brak cen dla tych dni — sprawdź nazewnictwo plików lub dane.")

minute_open = build_minute_open_series(prices_sel)
min_dt, max_dt = minute_open.index.min(), minute_open.index.max()
print("[minute_open] range:", min_dt, "→", max_dt, " (UTC)")

[dates] potrzeba dni UTC: ['2011-12-26', '2011-12-30', '2012-01-01', '2012-01-06', '2012-01-14', '2012-01-17', '2012-01-27', '2012-01-28', '2012-01-31', '2012-02-01'] … (łączna liczba: 2290 )
[prices/selected] files=1632 rows=515576 range=2011-12-26 22:00:00+00:00 → 2025-03-07 20:54:00+00:00 took 185.4s
[minute_open] range: 2011-12-26 22:00:00+00:00 → 2025-03-07 20:54:00+00:00  (UTC)


In [87]:
# ===== 3) Liczenie zmian =====
rows = []
t0 = time.time()
for r in tweets_sess.itertuples(index=False):
    dt_utc = pd.Timestamp(r.created_at)
    pct, base_open = pct_changes_for_tweet(minute_open, dt_utc, INTERVALS)
    if base_open is None:
        continue  # brak OPEN w minucie tweeta (np. mikroprzerwa) → pomiń
    row = {
        "tweet_id": str(r.tweet_id),
        "text": r.text,
        "created_at_local": pd.Timestamp(dt_utc).tz_convert(DISPLAY_TZ).strftime("%Y-%m-%d %H:%M:%S %Z"),
        "price_at_tweet_open": float(base_open),
        **{f"change_{m}m": pct[m] for m in INTERVALS}
    }
    rows.append(row)

res = pd.DataFrame(rows)
print(f"[done] wierszy w wyniku: {len(res)}  (obliczanie: {time.time()-t0:.1f}s)")


[done] wierszy w wyniku: 11222  (obliczanie: 10.5s)


In [88]:
# ===== 4) Podgląd w kolejności, o którą prosiłaś =====
if res.empty:
    print("Brak wyników (prawdopodobnie brak pokrycia cen w minucie tweeta).")
else:
    change_cols = [f"change_{m}m" for m in list(range(1,21)) + [30,60]]
    col_order = ["tweet_id","text","created_at_local","price_at_tweet_open"] + change_cols
    col_order = [c for c in col_order if c in res.columns]
    display(res[col_order].head(PREVIEW_N))

,tweet_id,text,created_at_local,price_at_tweet_open,change_1m,change_2m,change_3m,change_4m,change_5m,change_6m,...,change_13m,change_14m,change_15m,change_16m,change_17m,change_18m,change_19m,change_20m,change_30m,change_60m
0,910180847814197248,"@JimHeaney Absolutely. Moreover, we expect to ...",2017-09-19 18:36:39 CEST,125.52,-0.08,-0.08,-0.13,-0.15,-0.14,-0.10,...,-0.03,-0.02,-0.10,-0.01,-0.01,-0.03,-0.12,-0.09,-0.21,-0.29
1,910540337935826944,@Its_Subhamk Don't give up if the cause is imp...,2017-09-20 18:25:08 CEST,125.53,0.06,0.15,0.28,0.14,0.18,0.13,...,0.10,0.13,0.14,0.02,-0.02,0.00,-0.02,0.08,0.11,-0.05
2,912384568790999040,@Daimler Good,2017-09-25 20:33:27 CEST,114.60,-0.02,-0.02,-0.08,-0.05,0.03,-0.03,...,-0.15,-0.09,-0.15,-0.07,-0.22,-0.12,-0.12,-0.17,0.01,0.56
3,913489555663269888,@kevinroose Just another day in the office,2017-09-28 21:44:16 CEST,113.02,-0.01,0.15,0.12,0.12,-0.01,0.03,...,0.13,0.08,0.12,0.12,NaN,NaN,NaN,NaN,NaN,NaN
4,913489555663269889,@kevinroose Just another day in the office,2017-09-28 21:44:16 CEST,113.02,-0.01,0.15,0.12,0.12,-0.01,0.03,...,0.13,0.08,0.12,0.12,NaN,NaN,NaN,NaN,NaN,NaN
5,913775198423408640,@MeltingIce Assuming max acceleration of 2 to ...,2017-09-29 16:39:19 CEST,114.43,0.17,0.03,-0.03,0.00,-0.10,-0.08,...,-0.16,-0.14,-0.19,-0.19,-0.18,-0.21,-0.14,-0.04,0.05,-0.02
6,915939199718531072,@stapf The Tesla team has done this for many s...,2017-10-05 15:58:17 CEST,117.56,0.16,0.09,0.16,0.23,0.43,0.35,...,0.67,0.73,0.57,0.51,0.57,0.60,0.43,0.52,0.45,0.09
7,915957815197319168,@RickandMorty The singularity for this level o...,2017-10-05 17:12:15 CEST,117.41,0.04,-0.09,-0.20,-0.18,-0.14,-0.18,...,-0.07,-0.07,0.03,-0.01,0.03,0.09,0.01,0.09,0.20,-0.02
8,915988011602423808,@LoganJastremski I'm auditioning to be a roofi...,2017-10-05 19:12:14 CEST,117.90,-0.03,-0.06,0.00,-0.07,-0.06,-0.06,...,-0.02,0.04,0.03,0.13,0.10,0.11,0.19,0.21,0.08,0.30
9,915990261938786432,@RickandMorty I know that closet is nice and w...,2017-10-05 19:21:11 CEST,117.83,0.04,0.05,0.03,0.04,0.10,0.08,...,0.24,0.27,0.30,0.24,0.26,0.27,0.27,0.22,0.22,0.36


In [89]:
# ===== 5) Zapis do CSV =====
if not res.empty:
    output_csv_path = f"{DATA_DIR}/tweet_price_changes.csv"
    res.to_csv(output_csv_path, index=False)
    print(f"Zapisano wyniki do pliku: {output_csv_path}")
else:
    print("Brak wyników do zapisu.")

Zapisano wyniki do pliku: /content/drive/MyDrive/Thesis_v2/tweet_price_changes.csv
